In [1]:
import numpy as np
import pandas as pd

# GrCh37

Download RefSeq transcript annotations from [UCSC Table Browser](http://rohsdb.cmb.usc.edu/GBshape/cgi-bin/hgTables?hgsid=3960312_ZMTtI4bvavkuiWrNuR3OxAWB52dn&clade=mammal&org=Human&db=hg19&hgta_group=genes&hgta_track=ensGene&hgta_table=0&hgta_regionType=genome&position=chr21%3A33031597-33041570&hgta_outputType=primaryTable&hgta_outFileName=) using settings specified in the [wiki](https://github.com/keoughkath/ExcisionFinder/wiki/Get-gene-annotations). Alternatively, get this file [here](http://lighthouse.ucsf.edu/public_files_no_password/excisionFinderData_public/gene_annots/).

# GrCh38

Download RefSeq annotations from UCSC Table Browser using settings specified in the [wiki](https://github.com/keoughkath/ExcisionFinder/wiki/Get-gene-annotations). Alternatively, get this file [here](http://lighthouse.ucsf.edu/public_files_no_password/excisionFinderData_public/gene_annots/).

In [2]:
chroms = list(range(1,23)) + ['X','Y']

chroms_ = []

for chrom in chroms:
    chroms_.append('chr' + str(chrom))
    
chroms = chroms_

In [3]:
def get_gene_annots(infile):
    gene_df = pd.read_csv(infile, sep='\t',
                          usecols = ['name','chrom','txStart','txEnd',
                                    'cdsStart','cdsEnd','exonCount','exonStarts',
                                    'exonEnds','name2'])
    gene_df['size'] = gene_df['txEnd'] - gene_df['txStart']
    return gene_df

In [4]:
def filter_gene_annots(in_df):
    out_df = in_df[['name', 'chrom', 'txStart', 'txEnd', 'cdsStart', 'cdsEnd', 'exonCount',
       'exonStarts', 'exonEnds', 'size']].copy()
    out_df['official_gene_symbol'] = in_df['name2']

    gene_list_out = out_df.query('chrom in @chroms').dropna(axis=0).sort_values(by='size', ascending=False).groupby('official_gene_symbol').first()
    return gene_list_out

# GrCh37

Load gene annotations (input is the file you generated following the instructions above from the UCSC Table Browser).

In [10]:
hg19_in = get_gene_annots('ncbi_refseq_genes_hg19.txt')

Filter for canonical transcript.

In [11]:
gene_list_hg19 = filter_gene_annots(hg19_in)
print(len(gene_list_hg19))

27090


Save to file.

In [7]:
df = pd.read_csv('/pollard/home/kathleen/projects/AlleleAnalyzer/manuscript_analyses/1000genomes_analysis/get_gene_list/gene_list_hg19.tsv',
                sep="\t",
        header=0,
        names=[
            "name",
            "chrom",
            "txStart",
            "txEnd",
            "cdsStart",
            "cdsEnd",
            "exonCount",
            "exonStarts",
            "exonEnds",
            "size",
        ],
                )

In [8]:
df['ogs'] = df.index
df = df.reset_index(drop=True)

In [9]:
df.query('ogs == "BEST1"')

,name,chrom,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,size,ogs
1694,NR_134580.1,chr11,61717355,61731935,61731935,61731935,11,"61717355,61719242,61722578,61723189,61724315,6...","61717899,61719430,61722673,61723423,61724470,6...",14580,BEST1


In [49]:
gene_list_hg19.query('official_gene_symbol == "BEST1"')

,name,chrom,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,size
official_gene_symbol,,,,,,,,,,
BEST1,NR_134580.1,chr11,61717355,61731935,61731935,61731935,11,"61717355,61719242,61722578,61723189,61724315,6...","61717899,61719430,61722673,61723423,61724470,6...",14580


In [12]:
gene_list_hg19.to_csv('gene_list_hg19.tsv', sep='\t')

# save just gene names to another file

with open('genes_hg19.txt','w') as f:
    for gene in gene_list_hg19.index:
        f.write(gene + '\n')
        
with open('chroms_hg19.txt','w') as f:
    for chrom in gene_list_hg19['chrom'].tolist():
        f.write(chrom + '\n')

In [50]:
check = pd.read_hdf('/pollard/data/projects/AlleleAnalyzer_data/1kgp_data/hg19_analysis/1kgp_annotated_variants_by_chrom/chr11_annotated.h5',
                   'all', where='pos >= 61717355 and pos <= 61731935')

In [51]:
check.head()

,chrom,pos,ref,alt,makes_cpf1,breaks_cpf1,var_near_cpf1,makes_SpCas9,breaks_SpCas9,var_near_SpCas9,...,makes_SaCas9_KKH,breaks_SaCas9_KKH,var_near_SaCas9_KKH,makes_nmCas9,breaks_nmCas9,var_near_nmCas9,makes_cjCas9,breaks_cjCas9,var_near_cjCas9,region
180,11,61717400,T,C,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,True,region203295
181,11,61717412,A,G,False,False,True,False,False,True,...,False,False,True,False,False,False,False,False,True,region203295
182,11,61717444,C,G,False,False,False,False,True,True,...,False,False,True,False,False,False,False,False,True,region203295
183,11,61717508,C,T,False,False,False,False,True,True,...,False,False,False,False,False,False,False,False,False,region203295
184,11,61717552,G,A,False,False,False,False,True,True,...,False,False,True,False,False,True,False,False,False,region203295


# GrCh38 

Load gene annotations (input is the file you generated following the instructions above from the UCSC Table Browser).

In [28]:
hg38_in = get_gene_annots('ncbi_refseq_genes_hg38.txt')

Filter for canonical transcript.

In [33]:
gene_list_hg38 = filter_gene_annots(hg38_in)
print(len(gene_list_hg38))

38169


Save to file.

In [41]:
gene_list_hg38.to_csv('gene_list_hg38.tsv', sep='\t')

# save just gene names to another file

with open('genes_hg38.txt','w') as f:
    for gene in gene_list_hg38.index:
        f.write(gene + '\n')